***Preferencias en el Uso de Canales***

# ***Experimientos e iteraciones con los modelos***
Por Tatiana Molina - Carolina Alvarez

Una vez se realizó la limpieza de los datos la información resultante fue almacenada en un repositorio de Google Cloud para mejorar el rendimiento de la ejecucion de cada notebook para las siguientes ejecuciones.

# ***1. Consultar la base de datos de contactos y clientes***
Realizamos una lectura de los datos del repositorio para consultas de la organización (Google Cloud), al tratarse de datos confidenciales no es posible ejecutar estas celdas desde fuentes externas.

En este caso realizamos la consulta de la información resultante de la limpeiza de datos, sin embargo esta consulta se organizó para incluir unicamente el canal de contacto mas frecuente para cada cliente, esto dado que se identificó en las hiteraciones anteriores que un cliente podia tener varias contacto a traves de diferentes canales y por tanto esto puedia estar afectando el desempeño del modelo.

Se omitan la conexión y query a la base de datos por contener información sensible

In [ ]:
from google.cloud import bigquery

In [ ]:
import pandas as pd

In [ ]:
client = bigquery.Client(location="US")

In [ ]:
query = """
SELECT 
 """
query_job = client.query(query, location="US",) # API request - starts the query

In [ ]:
df = query_job.to_dataframe()

# ***2. Codificar y normalizar varaibles***



In [ ]:
#Cambiemos el tipo de datos de las variables para poder usar un codificación Label Encoder

df['descTema'] = df['descTema'].astype('str')
df['descCanalRadicacion'] = df['descCanalRadicacion'].astype('str')
df['descSegmentoAfiliado'] = df['descSegmentoAfiliado'].astype('str')
df['descCicloVida'] = df['descCicloVida'].astype('str')
df['descOcupacion'] = df['descOcupacion'].astype('str')
df['descRegional'] = df['descRegional'].astype('str')

In [ ]:
# Codificación de las variables

from sklearn.preprocessing import LabelEncoder # Codificación de etiquetas
LE = LabelEncoder()

df2 = df[['afi_hash64','descTema','descSexo', 'descSegmentoAfiliado', 'edadAfiliado', 
          'EstadoPO', 'EstadoPV', 'EstadoCES', 'ultimoIBC','IndicadorUsaClave', 'idAfiliadoTieneClave', 'TieneEmail',
          'descCicloVida','descOcupacion', 'descRegional' , 'descCanalRadicacion']]

df2 = df2[df2.descCanalRadicacion.isin(['LINEA DE SERVICIO', 'OFICINA DE SERVICIO', 'OFICINA VIRTUAL'])]

df2['afi_hash64'] = LE.fit_transform(df2['afi_hash64'])
df2["descTema"] = LE.fit_transform(df2['descTema'])
df2["descSexo"] = LE.fit_transform(df2['descSexo'])
df2["descSegmentoAfiliado"] = LE.fit_transform(df2['descSegmentoAfiliado'])
df2["EstadoPO"] = LE.fit_transform(df2['EstadoPO'])
df2["EstadoPV"] = LE.fit_transform(df2['EstadoPV'])
df2["EstadoCES"] = LE.fit_transform(df2['EstadoCES'])
df2["descCanalRadicacion"] = LE.fit_transform(df2['descCanalRadicacion'])
df2["IndicadorUsaClave"] = LE.fit_transform(df2['IndicadorUsaClave'])
df2["idAfiliadoTieneClave"] = LE.fit_transform(df2['idAfiliadoTieneClave'])
df2["TieneEmail"] = LE.fit_transform(df2['TieneEmail'])
df2["descCicloVida"] = LE.fit_transform(df2['descCicloVida'])
df2["descOcupacion"] = LE.fit_transform(df2['descOcupacion'])
df2["descRegional"] = LE.fit_transform(df2['descRegional'])

df2 = df2.drop(df.index[ (df['edadAfiliado'] > 90) & (df['edadAfiliado'] < 18)])


In [ ]:
#Normalización de datos
from sklearn.preprocessing import MinMaxScaler
import pandas as pd


scaler = MinMaxScaler()
scaler.fit(df2.iloc[:,1:15])
df_n= scaler.transform(df2.iloc[:,1:15])
df_n = pd.DataFrame(df_n, columns =  ['descTema','descSexo', 'descSegmentoAfiliado', 'edadAfiliado', 
          'EstadoPO', 'EstadoPV', 'EstadoCES', 'ultimoIBC','IndicadorUsaClave', 'idAfiliadoTieneClave', 'TieneEmail',
          'descCicloVida', 'descOcupacion','descRegional'])


### ***3. Modelo de redes neuronales***

Separamos los datos "X" de las etiquetas "y" para realizar una codificación One Hot Encoder requerida para el modelo

In [ ]:
X = df_n.to_numpy()

In [ ]:
y =df2['descCanalRadicacion'].to_numpy()

In [ ]:
#Codficación de etiquetas
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def oneHotEncoder(y):
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = y.reshape(len(y), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

In [ ]:
y = oneHotEncoder(y)

In [ ]:
# Partición de los datos en entrenamiento y pruebas
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=10, 
                                                    stratify =y )

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14882, 14), (3721, 14), (14882, 3), (3721, 3))

In [ ]:
# La libreria kers se usa como guía para la implementación del modelo
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

In [ ]:
# modelo secuencial con una capa densa con activación relu y  salida softmax
model = Sequential()
model.add(Dense(5, activation='relu', input_dim = 14))
model.add(Dense(3, activation='softmax'))

In [ ]:
# Función se usaremos para revertir la codificación de las etiquetas y conocer el canal predicho
def revierteOHE(y):
    y2=[]
    for i in range (len(y)):
        y2.append(np.argmax(y[i]))
    y2 = np.asarray(y2)
    return y2
  

In [ ]:
# la compilación del modelo se hace con los datos de test en 3 subsets

kf = KFold(n_splits = 3, shuffle = True,random_state = 0 )
precision = []
recall = []
F1 = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.compile(loss = "categorical_crossentropy", 
                 optimizer = "adam", 
                 metrics = ["accuracy"])
    
    model.fit(X_train, y_train, epochs = 100, batch_size = 100)
    
    y_pred = model.predict(X_test)
    y_pred = revierteOHE(y_pred)
    y_test = revierteOHE(y_test)
        
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average = 'macro')
    precision.append(prec)
    recall.append(rec)
    F1.append(f1)  

Epoch 1/100
125/125 [==============================] - 1s 2ms/step - loss: 1.1032 - accuracy: 0.3133
Epoch 2/100
125/125 [==============================] - 0s 2ms/step - loss: 1.0916 - accuracy: 0.3634
Epoch 3/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0866 - accuracy: 0.3786
Epoch 4/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0841 - accuracy: 0.3786
Epoch 5/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0826 - accuracy: 0.3786
Epoch 6/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0812 - accuracy: 0.3812
Epoch 7/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0736 - accuracy: 0.3986
Epoch 8/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0600 - accuracy: 0.4123
Epoch 9/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0510 - accuracy: 0.4389
Epoch 10/100
125/125 [==============================] - 0s 1ms/step - loss: 1.0450 - accura

In [ ]:
print("F1: " + str(np.mean(F1)))
print(F1)
print("\nprecision: " + str(np.mean(precision)))
print(precision)
print("\nrecall: " + str(np.mean(recall)))
print(recall)

F1: 0.47177129788104016
[0.4685482867065362, 0.47186796839716943, 0.4748976385394148]

precision: 0.5041361779592258
[0.4988242639285418, 0.505709032486963, 0.5078752374621726]

recall: 0.48598625275525614
[0.4833917448456401, 0.48554423007303066, 0.48902278334709764]


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 75        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 18        
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


***Mejor modelo:***
El mejor modelo hasta este punto sigue siendo el  KNN con 50 vecinos, weigh = distance y una selección de las 11 características más relevantes.

El modelo se redes neuronales no logra superar sus resultados, incluso la métrica queda muy por debajo con respecto al KNN.
